In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as model

# BASNET ATTEMPT 1


**Basic Architecture of BASNet consist of two Module, Prediction Module and Refined Module**

- lets start with prediction Module
- Construct helper function 
- The Prediction module is U-Net like structure (encoder-decoder)
- For Encoder we are going to use residual blocks



In [2]:
class basicBlock(tf.keras.Model):
    def __init__(self,filters,stride=1):
        super(basicBlock,self).__init__()
        self.conv1 = layers.Conv2D(filters,3,strides = stride,padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.Activation('relu')
        self.conv2 = layers.Conv2D(filters,3,strides=1,padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.Activation('relu')
        
    def call(self,x_in):
        res = x_in
        x = self.conv1(res)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = layers.add([x,res])
        x = self.relu2(x)
        
        return x
        

In [3]:
class reslayer1(tf.keras.Model):
    def __init__(self):
        super(reslayer1,self).__init__()
        self.convblock1 = basicBlock(64)
        self.convblock2 = basicBlock(64)
        self.convblock3 = basicBlock(64)
        
    def call(self,x_in):
        x = self.convblock1(x_in)
        x =  slef.convblock2(x)
        x = self.convblock3(x)
        return x

In [4]:
class reslayer2(tf.keras.Model):
    def __init__(self):
        super(reslayer2,self).__init__()
        self.convblock1 = basicBlock(128,2)
        self.convblock2 = basicBlock(128)
        self.convblock3 = basicBlock(128)
        self.convblock4 = basicBlock(128)
        
    def call(self,x_in):
        x = self.convblock1(x_in)
        x =  slef.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        return x

In [5]:
class reslayer3(tf.keras.Model):
    def __init__(self):
        super(reslayer3,self).__init__()
        self.convblock1 = basicBlock(256,2)
        self.convblock2 = basicBlock(256)
        self.convblock3 = basicBlock(256)
        self.convblock4 = basicBlock(256)
        self.convblock5 = basicBlock(256)
        self.convblock6 = basicBlock(256)
        
    def call(self,x_in):
        x = self.convblock1(x_in)
        x =  slef.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        return x

In [6]:
class reslayer4(tf.keras.Model):
    def __init__(self):
        super(reslayer4,self).__init__()
        self.convblock1 = basicBlock(512,2)
        self.convblock2 = basicBlock(512)
        self.convblock3 = basicBlock(512)
        
    def call(self,x_in):
        x = self.convblock1(x_in)
        x =  slef.convblock2(x)
        x = self.convblock3(x)
        return x

In [7]:
class RefUnet(tf.keras.Model):
    def __init__(self,filters):
        super(RefUnet,self).__init__()
        self.conv0 = layers.Conv2D(filters,3,padding='same')
        
        self.conv1 = layers.Conv2D(64,3,padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.Activation('relu')
        
        self.pool1=layers.MaxPool2D()
        
        self.conv2 = layers.Conv2D(64,3,padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.Activation('relu')
        
        self.pool2=layers.MaxPool2D()
        
        self.conv3 = layers.Conv2D(64,3,padding='same')
        self.bn3 = layers.BatchNormalization()
        self.relu3 = layers.Activation('relu')
        
        self.pool3=layers.MaxPool2D()
        
        self.conv4 = layers.Conv2D(64,3,padding='same')
        self.bn4 = layers.BatchNormalization()
        self.relu4 = layers.Activation('relu')
        
        self.pool4=layers.MaxPool2D()
        ######
        self.conv5 = layers.Conv2D(64,3,padding='same')
        self.bn5 = layers.BatchNormalization()
        self.relu5 = layers.Activation('relu')
        
        ####
        
        self.convd_4 = layers.Conv2D(64,3,padding='same')
        self.bnd_4 = layers.BatchNormalization()
        self.relud_4 = layers.Activation('relu')
        
        self.convd_3 = layers.Conv2D(64,3,padding='same')
        self.bnd_3 = layers.BatchNormalization()
        self.relud_3 = layers.Activation('relu')
        
        self.convd_2 = layers.Conv2D(64,3,padding='same')
        self.bnd_2 = layers.BatchNormalization()
        self.relud_2 = layers.Activation('relu')
        
        self.convd_1 = layers.Conv2D(64,3,padding='same')
        self.bnd_1 = layers.BatchNormalization()
        self.relud_1  = layers.Activation('relu')
        
        self.convd_0 = layers.Conv2D(1,3,padding='same')
        self.upscore2 = layers.UpSampling2D(2,interpolation='bilinear')
        
    def call(self,x):
        hx = x
        hx = self.conv0(hx)
        
        hx1 = self.relu1(self.bn1(self.conv1(hx)))
        hx = self.pool1(hx1)
        
        hx2 = self.relu2(self.bn2(self.conv2(hx)))
        hx = self.pool2(hx2)
        
        hx3 = self.relu3(self.bn3(self.conv3(hx)))
        hx = self.pool3(hx3)
        
        hx4 = self.relu4(self.bn4(self.conv4(hx)))
        hx = self.pool4(hx4)
        
        hx5 = self.relu5(self.bn5(self.conv5(hx)))
        
        
        hx = self.upscore2(hx5)
        
        d4 = self.relud_4(self.bnd_4(self.convd_4(layers.Concatenate(1)([hx,hx4]))))
        hx = self.upscore2(d4)
        
        d3 = self.relud_3(self.bnd_3(self.convd_3(layers.Concatenate(1)([hx,hx3]))))
        hx = self.upscore2(d3)
        
        d2 = self.relud_2(self.bnd_2(self.convd_2(layers.Concatenate(1)([hx,hx2]))))
        hx = self.upscore2(d2)
        
        d1 = self.relud_1(self.bnd_1(self.convd_1(layers.Concatenate(1)([hx,hx1]))))
        
        residual = self.convd_0(d1)
        
        return x + residual

In [24]:
class BasNet(tf.keras.Model):
    def __init__(self):
        super(BasNet,self).__init__()
        #------------Encoeder------------------
        
        self.inconv = layers.Conv2D(64,3,strides = 1,padding='same')
        self.inbn = layers.BatchNormalization()
        self.inrelu = layers.Activation('relu')
        
        #stage 1 
        self.encoder1 = reslayer1()#64
        #stage 2 
        self.encoder2 = reslayer2()#128
        #stage 3
        self.encoder3 = reslayer3()#256
        #stage 4
        self.encoder4 = reslayer4()#512
        
        self.pool4 = layers.MaxPool2D()
        
        #stage 5
        self.resb5_1 = basicBlock(512)
        self.resb5_2 = basicBlock(512)
        self.resb5_3 = basicBlock(512)
        
        self.pool5 = layers.MaxPool2D()
        
        #stage 6
        self.resb6_1 = basicBlock(512)
        self.resb6_2 = basicBlock(512)
        self.resb6_3 = basicBlock(512)
        
        
        #----------------Bridge------------------------
        #stage bridge
        self.convbg_1 = layers.Conv2D(512,3,dilation_rate=2)
        self.bnbg_1 = layers.BatchNormalization()
        self.relubg_1 = layers.Activation('relu')
        self.convbg_2 = layers.Conv2D(512,3,dilation_rate=2)
        self.bnbg_2 = layers.BatchNormalization()
        self.relubg_2 = layers.Activation('relu')
        self.convbg_3 = layers.Conv2D(512,3,dilation_rate=2)
        self.bnbg_3 = layers.BatchNormalization()
        self.relubg_3 = layers.Activation('relu')
        
        #-------------------Decoder----------------------
        
        #stage 6d
        self.convd6_1 = layers.Conv2D(512,3,padding='same')
        self.bnd6_1 = layers.BatchNormalization()
        self.relud6_1 = layers.Activation('relu')
        
        self.convd6_2 = layers.Conv2D(512,3,dilation_rate=2,padding='same')
        self.bnd6_2 = layers.BatchNormalization()
        self.relud6_2 = layers.Activation('relu')
        
        self.convd6_3 = layers.Conv2D(512,3,dilation_rate=2,padding='same')
        self.bnd6_3 = layers.BatchNormalization()
        self.relud6_3 = layers.Activation('relu')
        
        #stage 5d
        self.convd5_1 = layers.Conv2D(512,3,padding='same')
        self.bnd5_1 = layers.BatchNormalization()
        self.relud5_1 = layers.Activation('relu')
        
        self.convd5_2 = layers.Conv2D(512,3,padding='same')
        self.bnd5_2 = layers.BatchNormalization()
        self.relud5_2 = layers.Activation('relu')
        
        self.convd5_3 = layers.Conv2D(512,3,padding='same')
        self.bnd5_3 = layers.BatchNormalization()
        self.relud5_3 = layers.Activation('relu')
        
        #stage 4d
        self.convd4_1 = layers.Conv2D(512,3,padding='same')
        self.bnd4_1 = layers.BatchNormalization()
        self.relud4_1 = layers.Activation('relu')
        
        self.convd4_2 = layers.Conv2D(512,3,padding='same')
        self.bnd4_2 = layers.BatchNormalization()
        self.relud4_2 = layers.Activation('relu')
        
        self.convd4_3 = layers.Conv2D(256,3,padding='same')
        self.bnd4_3 = layers.BatchNormalization()
        self.relud4_3 = layers.Activation('relu')
        
        #stage 3d
        self.convd3_1 = layers.Conv2D(256,3,padding='same')
        self.bnd3_1 = layers.BatchNormalization()
        self.relud3_1 = layers.Activation('relu')
        
        self.convd3_2 = layers.Conv2D(256,3,padding='same')
        self.bnd3_2 = layers.BatchNormalization()
        self.relud3_2 = layers.Activation('relu')
        
        self.convd3_3 = layers.Conv2D(128,3,padding='same')
        self.bnd3_3 = layers.BatchNormalization()
        self.relud3_3 = layers.Activation('relu')
        
        #stage 2d
        self.convd2_1 = layers.Conv2D(128,3,padding='same')
        self.bnd2_1 = layers.BatchNormalization()
        self.relud2_1 = layers.Activation('relu')
        
        self.convd2_2 = layers.Conv2D(128,3,padding='same')
        self.bnd2_2 = layers.BatchNormalization()
        self.relud2_2 = layers.Activation('relu')
        
        self.convd2_3 = layers.Conv2D(64,3,padding='same')
        self.bnd2_3 = layers.BatchNormalization()
        self.relud2_3 = layers.Activation('relu')
        
        #stage 1d
        self.convd1_1 = layers.Conv2D(64,3,padding='same')
        self.bnd1_1 = layers.BatchNormalization()
        self.relud1_1 = layers.Activation('relu')
        
        self.convd1_2 = layers.Conv2D(64,3,padding='same')
        self.bnd1_2 = layers.BatchNormalization()
        self.relud1_2 = layers.Activation('relu')
        
        self.convd1_3 = layers.Conv2D(64,3,padding='same')
        self.bnd1_3 = layers.BatchNormalization()
        self.relud1_3 = layers.Activation('relu')
        
        #----------------------Bilinear upsampling-----------------
        self.upscore6 = layers.UpSampling2D(32,interpolation='bilinear')###
        self.upscore5 = layers.UpSampling2D(16,interpolation='bilinear')
        self.upscore4 = layers.UpSampling2D(8,interpolation='bilinear')
        self.upscore3 = layers.UpSampling2D(4,interpolation='bilinear')
        self.upscore2 = layers.UpSampling2D(2, interpolation='bilinear')
        
         ## -------------Side Output--------------
        self.outconvb = layers.Conv2D(1,3,padding=1)
        self.outconv6 = layers.Conv2D(1,3,padding=1)
        self.outconv5 = layers.Conv2D(1,3,padding=1)
        self.outconv4 = layers.Conv2D(1,3,padding=1)
        self.outconv3 = layers.Conv2D(1,3,padding=1)
        self.outconv2 = layers.Conv2D(1,3,padding=1)
        self.outconv1 = layers.Conv2D(1,3,padding=1)
        
        ## -------------Refine Module-------------
        self.refunet = RefUnet(64)
    
    def call(self,x):
        hx = x 
        #-----------------Encoder------------------
        hx = self.inconv(hx)
        hx = self.inbn(hx)
        hx = self.inrelu(hx)
        
        h1 = self.encoder1(hx)
        h2 = self.encoder2(h1)
        h3 = self.encoder3(h2)
        h4 = self.encoder4(h3)
        
        hx = self.pool4(h4)
        
        hx = self.resb5_1(hx)
        hx = self.resb5_2(hx)
        h5 = self.resb5_3(hx)
        
        hx = self.pool5(h5)
        
        hx = self.resb6_1(hx)
        hx = self.resb6_2(hx)
        h6 = self.resb6_3(hx)
        
        #-----------------------Bridge-------------------
        
        hx = self.relubg_1(self.bnbg_1(self.convbg_1(h6)))
        hx = self.relubg_2(self.bnbg_2(self.convbg_2(hx)))
        hbg = self.relubg_3(self.bnbg_3(self.convbg_3(hx)))
        
        #-----------------------Decoder--------------------
        
        hx = self.relud6_1(self.bnd6_1(self.convd6_1(layers.concatenate(1)([hbg,h6]))))
        hx = self.relud6_2(self.bnd6_2(self.convd6_2(hx)))
        hd6 = self.relud6_3(self.bnd6_3(self.convd6_3(hx)))
        
        hx = self.upscore2(hd6)
        
        hx = self.relud5_1(self.bnd5_1(self.convd5_1(layers.concatenate(1)([hbg,h5]))))
        hx = self.relud5_2(self.bnd5_2(self.convd5_2(hx)))
        hd5 = self.relud5_3(self.bnd5_3(self.convd5_3(hx)))
        
        hx = self.upscore2(hd5)
        
        hx = self.relud4_1(self.bnd4_1(self.convd4_1(layers.concatenate(1)([hbg,h4]))))
        hx = self.relud4_2(self.bnd4_2(self.convd4_2(hx)))
        hd4 = self.relud4_3(self.bnd4_3(self.convd4_3(hx)))
        
        hx = self.upscore2(hd4)
        
        hx = self.relud3_1(self.bnd3_1(self.convd3_1(layers.concatenate(1)([hbg,h3]))))
        hx = self.relud3_2(self.bnd3_2(self.convd3_2(hx)))
        hd3 = self.relud3_3(self.bnd3_3(self.convd3_3(hx)))
        
        hx = self.upscore2(hd3)
        
        hx = self.relud2_1(self.bnd2_1(self.convd2_1(layers.concatenate(1)([hbg,h2]))))
        hx = self.relud2_2(self.bnd2_2(self.convd2_2(hx)))
        hd2 = self.relud2_3(self.bnd2_3(self.convd2_3(hx)))
        
        hx = self.upscore2(hd2)
        
        hx = self.relud1_1(self.bnd1_1(self.convd1_1(layers.concatenate(1)([hbg,h1]))))
        hx = self.relud1_2(self.bnd1_2(self.convd1_2(hx)))
        hd1 = self.relud1_3(self.bnd1_3(self.convd1_3(hx)))
        
        #------------------------------side output------------------------
        
        db = self.outconvb(hbg)
        db = self.upscore6(db)
        
        d6 = self.outconv6(hd6)
        d6 = self.upscore6(d6)
        
        d5 = self.outconv5(hd5)
        d5 = self.upscore5(d5)
        
        d4 = self.outconv4(hd4)
        d4 = self.upscore4(d4)
        
        d3 = self.outconv3(hd3)
        d3 = self.upscore3(d3)
        
        d2 = self.outconv6(hd2)
        d2 = self.upscore6(d2)
        
        d1 = self.outconv1(hd1)
        
        #-----------------------------Refine Module--------------
        dout = self.refunet(hd1)
        
        return tf.sigmoid(dout), tf.sigmoid(d1), tf.sigmoid(d2), tf.sigmoid(d3), tf.sigmoid(d4), tf.sigmoid(d5), tf.sigmoid(d6), tf.sigmoid(db)

In [25]:
model = BasNet()

AttributeError: 'int' object has no attribute 'lower'

In [ ]:
model.build([None,224,224,3])
model.summary()